In [2]:
pip install sqlalchemy

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
import streamlit as st
from sqlalchemy import create_engine, Column, String, Integer, MetaData, Table, select
from sqlalchemy.orm import sessionmaker

# Set up the SQLite database
DATABASE_URL = "sqlite:///users.db"
engine = create_engine(DATABASE_URL)
metadata = MetaData()

# Define the User table
users_table = Table(
    'users', metadata,
    Column('id', Integer, primary_key=True, autoincrement=True),
    Column('username', String, unique=True, nullable=False),
    Column('password', String, nullable=False),
    Column('role', String, nullable=False)  # Role can be 'Student' or 'Evaluator'
)

# Create the table in the database
metadata.create_all(engine)

# Create a session to interact with the database
Session = sessionmaker(bind=engine)
session = Session()

# Function to register a new user
def register_user(username, password, role):
    # Check if the user already exists
    existing_user = session.execute(select(users_table).where(users_table.c.username == username)).fetchone()
    if existing_user:
        st.warning("Username already exists")
    else:
        # Insert new user into the database
        session.execute(users_table.insert().values(username=username, password=password, role=role))
        session.commit()
        st.success(f"Successfully registered as a {role}")

# Function to authenticate user login
def authenticate_user(username, password, role):
    user = session.execute(select(users_table).where(users_table.c.username == username).where(users_table.c.password == password).where(users_table.c.role == role)).fetchone()
    return user is not None

# Function to handle login and registration
def login_page():
    st.title("Login")

    role = st.selectbox("Select your role", ["Student", "Evaluator"])
    action = st.radio("Action", ["Login", "Register"])

    username = st.text_input("Username")
    password = st.text_input("Password", type="password")

    if st.button("Submit"):
        if action == "Register":
            register_user(username, password, role)
        else:
            if authenticate_user(username, password, role):
                st.session_state["user"] = {"role": role, "username": username}
                st.success("Login successful")
            else:
                st.error("Invalid credentials")

# Sidebar navigation
def sidebar():
    st.sidebar.title("Navigation")
    if "user" not in st.session_state:
        st.sidebar.write("Please log in to continue")
        return
    role = st.session_state["user"]["role"]
    st.sidebar.write(f"Logged in as: {role}")

    if role == "Student":
        page = st.sidebar.radio("Go to", ["About", "Student", "Score", "Future Scope", "Logout"])
    elif role == "Evaluator":
        page = st.sidebar.radio("Go to", ["About", "Evaluator", "Score", "Future Scope", "Logout"])
    else:
        page = st.sidebar.radio("Go to", ["About", "Future Scope", "Logout"])

    if page == "Logout":
        del st.session_state["user"]
        st.experimental_rerun()

    return page

# Main application logic
def main():
    if "user" not in st.session_state:
        login_page()
    else:
        page = sidebar()

        if page == "About":
            about_page()
        elif page == "Student":
            student_page()
        elif page == "Evaluator":
            evaluator_page()
        elif page == "Score":
            score_page()
        elif page == "Future Scope":
            future_scope_page()

if __name__ == "__main__":
    main()


2024-08-24 16:38:47.334 WARNING streamlit.runtime.state.session_state_proxy: Session state does not function when running a script without `streamlit run`
2024-08-24 16:38:47.559 
  command:

    streamlit run /home/manax/.local/lib/python3.10/site-packages/ipykernel_launcher.py [ARGUMENTS]


In [5]:
streamlit run /home/manax/.local/lib/python3.10/site-packages/ipykernel_launcher.py

SyntaxError: invalid syntax (2665882438.py, line 1)